In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10 # type: ignore
from tensorflow.keras.models import Sequential,Model # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # type: ignore
from tensorflow.keras.utils import to_categorical  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
     